In [6]:
!pip install ultralytics opencv-python-headless lap tqdm --quiet
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.4 MB/s eta 0:00:00:00:010:01


In [7]:
import os

# Set YOLOv8 to quiet mode
os.environ['YOLO_VERBOSE'] = 'False'

In [8]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import torch
import pandas as pd
from tqdm import tqdm

def calculate_distance(p1, p2):
    """Calculate Euclidean distance between two points."""
    return np.linalg.norm(np.array(p1) - np.array(p2))

def load_model(model_path='yolov.pt', device=None):
    """Load the YOLOv8 model and move it to the specified device."""
    model = YOLO(model_path,verbose=False)
    if device:
        model.to(device)
    return model


In [9]:
def process_video(video_path, model, tracker, skip_frames=1, resize=None):
    """
    Process a video to detect and track objects, calculating their distances and speeds.

    Parameters:
    - video_path: Path to the input video file.
    - model: Preloaded YOLOv8 model.
    - tracker: Initialized DeepSort tracker.
    - skip_frames: Number of frames to skip between processing.
    - resize: Tuple (width, height) to resize frames, or None to keep original size.

    Returns:
    - players_data: Dictionary containing tracking data for players and balls.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second

    prev_positions = {}  # Store previous positions to calculate distance
    players_data = {}  # Dictionary to store player and ball data

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames to reduce computation
        if frame_count % skip_frames != 0:
            frame_count += 1
            continue
        frame_count += 1

        # Resize frame if specified
        if resize is not None:
            frame = cv2.resize(frame, resize)

        # Perform object detection
        results = model(frame)
        boxes = results[0].boxes

        detections = []
        if boxes:
            # Batch processing to minimize data transfer
            boxes_data = boxes.xyxy.cpu().numpy()
            confs = boxes.conf.cpu().numpy()
            classes = boxes.cls.cpu().numpy().astype(int)

            # Filter for 'person' and 'sports ball'
            mask = np.isin(classes, [0,1,2,3])
            boxes_data = boxes_data[mask]
            confs = confs[mask]
            classes = classes[mask]

            # Prepare detections for DeepSort
            for i in range(len(boxes_data)):
                x1, y1, x2, y2 = boxes_data[i]
                conf = confs[i]
                cls = classes[i]

                # Convert bbox to [x, y, w, h] format
                x = x1
                y = y1
                w = x2 - x1
                h = y2 - y1
                bbox = [x, y, w, h]

                detection = (bbox, conf, cls)  # Correct format
                detections.append(detection)

        # Update tracker with current frame detections
        tracks = tracker.update_tracks(detections, frame=frame)  # Pass frame parameter

        # Process each track
        for track in tracks:
            if not track.is_confirmed():
                continue
            track_id = track.track_id
            bbox = track.to_ltwh(orig=True)  # [left, top, width, height]
            center = (bbox[0] + bbox[2] / 2, bbox[1] + bbox[3] / 2)

            # Retrieve class ID from track.det_class
            cls = track.det_class
            if cls == 1:
                object_type = 'goalkeeper'
            elif cls == 2:
                object_type = 'player'
            elif cls == 3:
                object_type = 'referee'
            elif cls == 0:
                object_type = 'ball'
            else:
                object_type = 'unknown'

            # Calculate distance and speed
            if track_id in prev_positions:
                distance_traveled = calculate_distance(prev_positions[track_id], center)
                speed = distance_traveled * fps  # Speed = distance per second
                # Update existing record
                if track_id in players_data:
                    players_data[track_id]['distance'] += distance_traveled
                    players_data[track_id]['speed'] = speed
                else:
                    # Add new record
                    players_data[track_id] = {
                        'distance': distance_traveled,
                        'speed': speed,
                        'type': object_type,
                    }
            else:
                # Initialize new track data
                players_data[track_id] = {
                    'distance': 0.0,
                    'speed': 0.0,
                    'type': object_type,
                }

            # Update previous position
            prev_positions[track_id] = center

    cap.release()
    return players_data


In [10]:
import os
import json

def get_video_paths_in_folder(input_folder):
    video_paths = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".mp4"):  # Filter only video files
                video_paths.append(os.path.join(root, file))
    return video_paths

In [11]:
input_folder= '/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0'
video_paths = get_video_paths_in_folder(input_folder)
video_paths1,video_paths2 = video_paths[:38],video_paths[38:]
print(len(video_paths1),len(video_paths2))

38 38


In [14]:
import logging

# Set logging level to WARNING to suppress INFO logs
logging.getLogger('ultralytics').setLevel(logging.WARNING)


def process_videos(video_paths, output_json_path):
    """
    Processes a list of video files and saves the results to a JSON file.

    Args:
        video_paths (list): List of paths to video files.
        output_json_path (str): Path to the JSON file to save results.
    """
    results = []

    for video_path in tqdm(video_paths):
        print(f"Processing video: {video_path}")
        
        # Call the process_video function
        video_data = process_video(video_path, model, skip_frames=1, resize=None, tracker=DeepSort(max_age=450))
        
        # Include the video name in the result
        video_data['video_name'] = os.path.basename(video_path)
        
        # Append the processed data to the results list
        results.append(video_data)

    # Write results to the JSON file
    with open(output_json_path, 'w') as json_file:
        json.dump(results, json_file, indent=4)
    print(f"Results have been saved to {output_json_path}")


# Usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = load_model('/kaggle/input/yolov8-finetuned/pytorch/default/2/best.pt', device=device)

output_json_path = '/kaggle/working/processed_videos.json'

# Process the videos and save results to JSON
process_videos(video_paths2, output_json_path)

  0%|          | 0/38 [00:00<?, ?it/s]

Processing video: /kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (6).mp4


  3%|▎         | 1/38 [03:39<2:15:14, 219.31s/it]

Processing video: /kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (48).mp4


  5%|▌         | 2/38 [07:21<2:12:37, 221.05s/it]

...

Processing video: /kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (50).mp4


 97%|█████████▋| 37/38 [2:02:56<03:19, 199.93s/it]

Processing video: /kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (47).mp4


100%|██████████| 38/38 [2:06:26<00:00, 199.63s/it]

Results have been saved to /kaggle/working/processed_videos.json
